In [1]:
%matplotlib inline
import sys
import xarray as xr
import numpy as np
import os
import yaml
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom, Variable

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source')

from OP_functions import *
from OP_Kernels_e3t import *

local = 0
restart=0

In [2]:
def get_timestamps(start,length):
    timestamps=[]
    duration = timedelta(days=length)
    for day in range(duration.days):
        timestamps.append([start + timedelta(days=day)])
    return np.array(timestamps, dtype='datetime64')

def find_temp(rootdir):
    dirs=[]
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            dirs.append(d)
    temp=sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[:1][0]
    return temp[-12:]

def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)


In [3]:
# Define paths
paths = path(local)
#Load configuration from .yaml
param = load_config(['/home/jvalenti/MOAD/analysis-jose/OParcels/yaml/test.yaml'])
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N0 = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
distr = param['param']['distr']
MFc = param['param']['MFc']

In [4]:
#Set deploy coordinates following yaml   

clat = param['param']['lats']
clon = param['param']['lons']
#clon, clat = [float(outf_lon)],[float(outf_lat)] 
duration = timedelta(days=length)
#Set deploy locations
if distr == 'hmg':
    clat,clon = p_unidist(N0,N0)
    N = len(clat)
elif distr == 'trst':
    clat,clon = transect_deploy(clat,clon,N0)
    N = N0
elif distr == 'std':
    N = len(param['param']['lats'])
elif distr == 'pd':
    clat, clon, N = pandas_deploy(N0,MFc,int(dtp))
    n = 1

x_offset, y_offset, z = p_deploy(N,n,dmin,dd,rrr)

lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])

In [5]:
#Set start date time and the name of the output file

daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)

In [18]:
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist,local)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

#Find file names and variable names
varlist=['US','VS','WL','Diat','Flag','R','T','S','FS','ssh','Bathy','Kz']
filenames,variables=filename_set(start,length,varlist,local)

#Add Stokes Drift fields
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

#Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Kz)

#Add fields located at node T
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'deptht','time': 'time_counter'}
Diat = Field.from_netcdf(filenames['Diat'], variables['Diat'], dimensions,allow_time_extrapolation=True)
Flag = Field.from_netcdf(filenames['Flag'], variables['Flag'], dimensions,allow_time_extrapolation=True)
R = Field.from_netcdf(filenames['R'], variables['R'], dimensions,allow_time_extrapolation=True)
S = Field.from_netcdf(filenames['S'], variables['S'], dimensions,allow_time_extrapolation=True)
T = Field.from_netcdf(filenames['T'], variables['T'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Diat)
field_set.add_field(Flag)
field_set.add_field(R)
field_set.add_field(S)
field_set.add_field(T)

#Add Bathymetry 2D field
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

#Add SSH and Rivers 2D fields
dimensions = {'lon': 'glamt', 'lat': 'gphit','time': 'time_counter'}
cms = Field.from_netcdf(filenames['ssh'], variables['ssh'], dimensions,allow_time_extrapolation=True)
field_set.add_field(cms)
Fraser = Field.from_netcdf(filenames['FS'], variables['FS'], dimensions,allow_time_extrapolation=True,timestamps=get_timestamps(start,length))
field_set.add_field(Fraser)

####BUILD Particle typeN######
MPParticle = particle_maker(param)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [19]:
if restart==1:
    name_temp=find_temp(paths['out'])
    os.system(f"cd {paths['out']} && parcels_convert_npydir_to_netcdf {name_temp}")
    outfile=newest(paths['out'])
    pset = ParticleSet.from_particlefile(field_set, MPParticle,outfile)
else:
    if dtp == 0:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))
    else:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=dtp))

In [22]:
def kernel_asem(pset,config):
    KER = Advection
    if 'Buoyancy' in config['kernel']:
        KER += pset.Kernel(Buoyancy)
    if 'Stokes_drift' in config['kernel']:
        KER += pset.Kernel(Stokes_drift)
    if 'Beaching' in config['kernel']:
        KER += pset.Kernel(Beaching)
        KER += pset.Kernel(Unbeaching)
    if 'Turb_mix' in config['kernel']:
        KER += pset.Kernel(turb_mix)
    if 'Biofilm' in config['kernel']:
        KER += pset.Kernel(Biofilm)
    if 'Stokes_driftRK4_3D' in config['kernel']:
        KER += pset.Kernel(Stokes_driftRK4_3D)
    
    return KER

In [28]:
KERNELS =  Advection + pset.Kernel(Buoyancy) + pset.Kernel(Stokes_drift) + pset.Kernel(turb_mix) + pset.Kernel(Displacement) + pset.Kernel(Unbeaching)
pset.execute(KERNELS,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

AttributeError: Particle type PType<ArrayMPParticle>::[PVar<lon|<class 'numpy.float64'>>, PVar<lat|<class 'numpy.float64'>>, PVar<depth|<class 'numpy.float64'>>, PVar<time|<class 'numpy.float64'>>, PVar<id|<class 'numpy.int64'>>, PVar<dt|<class 'numpy.float64'>>, PVar<_next_dt|<class 'numpy.float64'>>, PVar<once_written|<class 'numpy.int32'>>, PVar<state|<class 'numpy.int32'>>, PVar<ro|<class 'numpy.float32'>>, PVar<diameter|<class 'numpy.float32'>>, PVar<SDD|<class 'numpy.float32'>>, PVar<SDL|<class 'numpy.float32'>>, PVar<length|<class 'numpy.float32'>>, PVar<Lb|<class 'numpy.float32'>>, PVar<Db|<class 'numpy.float32'>>, PVar<Ub|<class 'numpy.float32'>>, PVar<beached|<class 'numpy.float32'>>, PVar<tau|<class 'numpy.float32'>>, PVar<fratio|<class 'numpy.float32'>>, PVar<Nbac|<class 'numpy.float32'>>, PVar<Nflag|<class 'numpy.float32'>>, PVar<Kh|<class 'numpy.float32'>>, PVar<dtmax|<class 'numpy.float32'>>, PVar<ngrids|<class 'numpy.int32'>>, PVar<xi|<class 'numpy.int32'>>, PVar<yi|<cl